# Computing Partials and Amplitudes 

In [ ]:
import librosa
import librosa.display
import numpy as np
import os
from IPython import display as ipd
from scipy.io import wavfile
import matplotlib.pyplot as plt
import scipy
import pandas as pd

from libsoni.core.methods import generate_tone_additive_synthesis
from libsoni.util.utils import normalize_signal


%matplotlib inline

In [ ]:
def pitch_to_frequency(pitch):
    return  440.0  * 2 ** ((pitch - 69) / 12)
def get_pitch_from_samplename(sample):
    return int(sample[7:10])
def get_folder_from_samplename(sample):
    return sample[:2]

In [ ]:
# loading the audio sample
Sample = '02Vio1F069m_np___0.wav'
SAMPLE_DIR = os.path.join('..','SNDB',get_folder_from_samplename(Sample), Sample)

## Computing Partials

In [ ]:
pitch_sample = get_pitch_from_samplename(Sample)
f_sample = pitch_to_frequency(pitch_sample)
x, Fs = librosa.load(SAMPLE_DIR)

x = x[:3*Fs]
x = normalize_signal(x)
print('loaded sample:')
ipd.display(ipd.Audio(x, rate=Fs))

## Performing FFT

In [ ]:
# Define Length of FFT
N_fft = 4096*8

# Perform FFT to obtain the frequency domain representation
X = np.fft.fft(x, n = N_fft, norm='ortho')

# Calculate the magnitudes of the FFT coefficients
magnitude = np.abs(X)

# Find the frequencies corresponding to each FFT coefficient
frequencies = np.fft.fftfreq(len(magnitude), 1 / Fs)

phase = np.angle(X)

# Keep only the positive frequencies
frequencies = frequencies[:len(frequencies) // 2]
magnitude = magnitude[:len(magnitude) // 2]
phase = phase[:len(phase) // 2]

plt.figure(figsize=(8, 4))
plt.plot(frequencies,magnitude)
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude')
plt.title('FFT for %s' %Sample )
#plt.legend()
plt.grid(True)
plt.show()
plt.show()

In [ ]:
# Define number of peaks to encounter for partials
N = 100

# Find N peaks in FFT
ind = np.argpartition(magnitude, -N)[-N:]

# Get parameters for synthesis
frequencies_synthesis = frequencies[ind]
magnitudes_synthesis = magnitude[ind]
phases_synthesis = phase[ind]

partials = frequencies_synthesis / f_sample
partials_amplitude = magnitudes_synthesis
partials_phase_offsets=phases_synthesis


# perform synthesis
resyn_1 = generate_tone_additive_synthesis(pitch=pitch_sample,
                                       partials=partials,
                                       partials_amplitudes=partials_amplitude,
                                       partials_phase_offsets=partials_phase_offsets,
                                       duration_sec = 3.0)

resyn_1 = normalize_signal(resyn_1)

print('Re-synthesized sample using %s peaks of fft:' %N)
ipd.display(ipd.Audio(resyn_1, rate=Fs))

In [ ]:
for N in [1,5,10,50,100,200]:
    # Find N peaks in FFT
    ind = np.argpartition(magnitude, -N)[-N:]
    
    # Get parameters for synthesis
    frequencies_synthesis = frequencies[ind]
    magnitudes_synthesis = magnitude[ind]
    phases_synthesis = phase[ind]
    
    partials = frequencies_synthesis / f_sample
    partials_amplitude = magnitudes_synthesis
    partials_phase_offsets=phases_synthesis

    # perform synthesis
    resyn_N = generate_tone_additive_synthesis(pitch=pitch_sample,
                                           partials=partials,
                                           partials_amplitudes=partials_amplitude,
                                           partials_phase_offsets=partials_phase_offsets,
                                           duration_sec = 3.0)
    
    resyn_N = normalize_signal(resyn_N)
    
    print('Re-synthesized Sample using %s peaks of fft:' %N)
    ipd.display(ipd.Audio(resyn_N, rate=Fs))

## Different Approach: Search Peaks in Octave-Subbands

In [ ]:
# Define the number of octave bands
num_octave_bands = 12
num_peaks_per_octave_band = 1
print('#sinusoids: %s' % int(num_octave_bands*num_peaks_per_octave_band))
# Calculate the center frequencies of the octave bands
center_frequencies = [f_sample * i for i in range(1, num_octave_bands)]


plt.figure(figsize=(8, 4))

partials = []
partials_amplitude = []
partials_phase_offsets = []

for center_freq in center_frequencies:

    lower_freq = center_freq - f_sample / 2
    upper_freq = center_freq + f_sample / 2
        
    band_indices = np.where((frequencies >= lower_freq) & (frequencies < upper_freq))
 
    frequencies_band = frequencies[band_indices]
    magnitude_band = magnitude[band_indices]
    phases_band = phase[band_indices]

    plt.axvline(lower_freq, color='r', linestyle='--', alpha=0.5)
    plt.axvline(upper_freq, color='r', linestyle='--', alpha=0.5)
    plt.plot(frequencies[band_indices],magnitude[band_indices])

    ind = np.argpartition(magnitude_band, -num_peaks_per_octave_band)[-num_peaks_per_octave_band:]

    partials.append(frequencies_band[ind][0]/f_sample)
    partials_amplitude.append(magnitude_band[ind][0])
    partials_phase_offsets.append(phases_band[ind][0])
    

resyn_2 = generate_tone_additive_synthesis(pitch=pitch_sample,
                                           partials=partials,
                                           partials_amplitudes=partials_amplitude,
                                           partials_phase_offsets=partials_phase_offsets,
                                           duration_sec = 3.0)

resyn_2 = normalize_signal(resyn_2)

ipd.display(ipd.Audio(resyn_2, rate=Fs))
plt.plot()
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude')
plt.title('Sub-Octave Bands')
plt.grid(True)
plt.show()

what's over?

In [ ]:
x = normalize_signal(x)
resyn_2 = normalize_signal(resyn_2)

plt.plot(x-resyn_2)
ipd.display(ipd.Audio((x-resyn_2),rate=Fs))

In [ ]:

from libsoni.core.pianoroll import sonify_pianoroll_additive_synthesis, \
                                   sonify_pianoroll_fm_synthesis, \
                                   sonify_pianoroll_clicks, \
                                   sonify_pianoroll_sample, \
                                   visualize_pianoroll

from libsoni.core.tse import sonify_tse_clicks
from libsoni.util.utils import mix_sonification_and_original, get_preset

bach_df = pd.read_csv(os.path.join('data_csv',
                                   'demo_pianoroll',
                                   'FMP_C1_F12_Bach_BWV846_Sibelius-Tracks.csv'),delimiter=';')

bach_audio, _ = librosa.load(os.path.join('data_audio',
                                          'demo_pianoroll',
                                          'FMP_C1_F12_Bach_BWV846_Sibelius-Tracks.wav'))

visualize_pianoroll(bach_df, figsize=(10, 7), colors='gist_rainbow', title='Piano Roll: Fugue in C Major, J. S. Bach');

# Additive Synthesis

sonified_bach_as = sonify_pianoroll_additive_synthesis(pianoroll_df=bach_df,
                                                       partials=partials,
                                                       partials_amplitudes=partials_amplitude,
                                                       partials_phase_offsets=partials_phase_offsets)

sonified_bach_as_w_original = mix_sonification_and_original(sonification=sonified_bach_as,
                                                            original_audio=bach_audio,
                                                            gain_lin_original_audio=0.25,
                                                            panning=0)


print('Sonified with Additive Synthesis:')
ipd.display(ipd.Audio(sonified_bach_as, rate=Fs))


print('Sonified with Additive Synthesis:')
ipd.display(ipd.Audio(sonified_bach_as_w_original, rate=Fs))


In [ ]:
def resynthesize(sample, num_octave_bands = 12, num_peaks_per_octave_band = 1, target_sample=69):
    SAMPLE_DIR = os.path.join('..','SNDB',get_folder_from_samplename(sample), sample)
    
    pitch_sample = get_pitch_from_samplename(sample)
    
    f_sample = pitch_to_frequency(pitch_sample)
    x, Fs = librosa.load(SAMPLE_DIR)
    # Define Length of FFT
    N_fft = 4096*8
    
    # Perform FFT to obtain the frequency domain representation
    X = np.fft.fft(x, n = N_fft, norm='ortho')
    
    # Calculate the magnitudes of the FFT coefficients
    magnitude = np.abs(X)
    
    # Find the frequencies corresponding to each FFT coefficient
    frequencies = np.fft.fftfreq(len(magnitude), 1 / Fs)
    
    phase = np.angle(X)
    
    
    frequencies = frequencies[:len(frequencies) // 2]
    magnitude = magnitude[:len(magnitude) // 2]
    phase = phase[:len(phase) // 2]
    # Calculate the center frequencies of the octave bands
    center_frequencies = [f_sample * i for i in range(1, num_octave_bands)]
    
    partials = []
    partials_amplitude = []
    partials_phase_offsets = []
    
    for center_freq in center_frequencies:
    
        lower_freq = center_freq - f_sample / 2
        upper_freq = center_freq + f_sample / 2
            
        band_indices = np.where((frequencies >= lower_freq) & (frequencies < upper_freq))
     
        frequencies_band = frequencies[band_indices]
        magnitude_band = magnitude[band_indices]
        phases_band = phase[band_indices]
    
        ind = np.argpartition(magnitude_band, -num_peaks_per_octave_band)[-num_peaks_per_octave_band:]
    
        partials.append(frequencies_band[ind][0]/f_sample)
        partials_amplitude.append(magnitude_band[ind][0])
        partials_phase_offsets.append(phases_band[ind][0])
        
    
    resyn_2 = generate_tone_additive_synthesis(pitch=target_sample,
                                               partials=partials,
                                               partials_amplitudes=partials_amplitude,
                                               partials_phase_offsets=partials_phase_offsets,
                                               duration_sec = 3.0)
    
    return normalize_signal(resyn_2)

In [ ]:
samples = [
    '01Pia1F069m_np___0.wav',
    '02Vio1F069m_np___0.wav',
    '03Cel1F069m_np___0.wav',
    '04Con1F060m_np___0.wav',
    '05Tru1M069m_np___0.wav',
    '06Tro1F069m_np___0.wav',
    '07Hor1F069m_np___0.wav',
    '08Obo1F069m_np___0.wav',
    '09Bas1F069m_np___0.wav',
    '10Cla1F069m_np___0.wav',
    '11Flu1F069m_np___0.wav'
    ]

for sample in samples:
    print(sample[2:5])
    x = resynthesize(sample, num_octave_bands = 10, num_peaks_per_octave_band = 1, target_sample= 60)
    ipd.display(ipd.Audio(x, rate=Fs))